In [1]:
## INSTALL LIBRARIES NOT INCLUDED BY DEFAULT
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install pillow
!{sys.executable} -m pip install torchvision
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install ordereddict

from PIL import Image
sys.modules['Image'] = Image 

from PIL import Image

import Image


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from glob import glob
import os
from torchvision import models
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
import time
from collections import OrderedDict
import collections
from torch.utils.data import Dataset
from shutil import copyfile
%matplotlib inline


from torchvision import transforms

from torch import optim


In [21]:
## The pytorch image classifier works by creating different collections of files for Training, Testing and Validation 
## each stored within their own directory.  
## First check if they exist and if not create them. 
## Next load the data file and create a dataframe for use.  Set the dataframe's index value to the column with the Unique
## identifiers for each row of data

for i in ['train', 'valid','test']:
    if not os.path.exists(os.path.join('scadams/tmpf/', i)):
        print("making directory")
        os.mkdir(os.path.join('scadams/tmpf/', i))
    else:
        print("directory already exists")

labels_path = 'Project/Labels_Full2.csv'

data = pd.read_csv(labels_path)
labels_df = pd.DataFrame(data)
labels_df=labels_df.set_index('imdb_title_id')


directory already exists
directory already exists
directory already exists


In [4]:
### Split the dataset into 80% Train 10% Test 10% Validation
threshold=[int(len(labels_df)*.8),int(len(labels_df)*.9)]
train,val,test=np.split(labels_df,threshold)

print(len(train),len(val),len(test))

16096 2012 2012


In [5]:
## For each value in each dataset, move the files from the source directory with all images to the respective folder
## NOTE the copying of file only needs to be done once as long as the distribution does not change.

train_file_names = []
for i in range(0, train.index.size):

    file_name = train.take([i]).index[0]+'.jpg'
    train_file_names.append(file_name)
   # copyfile(os.path.join('FullImages/',file_name),os.path.join('tmp/train/',file_name))

valid_file_names = []
for i in range(0, val.index.size):

    file_name = val.take([i]).index[0]+'.jpg'
    valid_file_names.append(file_name)
   # copyfile(os.path.join('FullImages/',file_name),os.path.join('tmp/valid/',file_name))

test_file_names = []
for i in range(0, test.index.size):

    file_name = test.take([i]).index[0]+'.jpg'
    test_file_names.append(file_name)

  #  copyfile(os.path.join('FullImages/',file_name),os.path.join('tmp/test/',file_name))

In [6]:
## Define the class that will be used to hold the custom Dataset. 
## Create an item that includes both the image reference (as a tensor) and its coorosponding correct labels

class MultiClassImages(Dataset):
    def __init__(self, dataframe, folder_dir, transform = None):
        self.dataframe = dataframe
        self.folder_dir = folder_dir
        self.transform = transform
    
        self.file_names = dataframe.index### This field does not include .jpg
        self.labels = dataframe.values.tolist()

    def __len__(self):

        return (len(self.dataframe))

    def __getitem__(self, index):
        image = Image.open(os.path.join(self.folder_dir, self.file_names[index]+'.jpg')).convert('RGB') 
    #    processed_image = vgg16.preprocess_input(numpy_image.copy()) # need deep-copy as it will change the image_batch
        #label = self.labels[index][0]
        label = self.labels[index]

        sample = {'image': image, 'label': np.array(label).astype(np.float)}
                  


        if self.transform:
            image = self.transform(sample['image'])

            sample = {'image': image, 'label': np.array(label).astype(np.float)}

            return sample

In [22]:
## Define the transformation that is applied to each image
## Resize to 256 x 256
## Crop from centered to be 224 x 224
## Convert to Tensor
## Apply Imagenet Normalization values

tfms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224),transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
  
train_dl = MultiClassImages(train, 'scadams/tmpf/train/', transform = tfms)
valid_dl = MultiClassImages(val, 'scadams/tmpf/valid/', transform = tfms)
test_dl = MultiClassImages(test, 'scadams/tmpf/test/', transform = tfms)
        

In [8]:
## Create a data loader object for each of the datasets
train_dataloader = torch.utils.data.DataLoader(train_dl, shuffle = True, batch_size = 64, num_workers = 3)
valid_dataloader = torch.utils.data.DataLoader(valid_dl, shuffle = True, batch_size = 64, num_workers = 3)
test_dataloader = torch.utils.data.DataLoader(test_dl, shuffle = False, batch_size = 1, num_workers = 3)

In [9]:
## Load the Pretrained Resnet50 model
##Add a fully connected layer to change the features from 2048 to 20
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
model.fc = torch.nn.Linear(2048, 20)

Using cache found in /home/scadams/.cache/torch/hub/pytorch_vision_v0.6.0


In [10]:
## Check if a GPU is being used

def check_cuda():
    _cuda = False
    if torch.cuda.is_available():
        _cuda = True
    return _cuda
is_cuda = check_cuda()
print(is_cuda)
if is_cuda:
    model.cuda()


False


In [11]:
## Define code to Fit the Model. 
## Loss function is BCEWithLogitLoss
## Optimizer is Adam
## For debugging purpose every iteration within patch is printed to show status

from pprint import pprint
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())
##optimizer = optim.SGD(model.parameters(),  lr = 0.01, momentum = 0.9, weight_decay = 1e-5)
def fit_model(epochs, model, dataloader, phase = 'train', volatile = False):
    
    pprint("Epoch: {}".format(epochs))
    running_loss = []
    running_acc = []
    
    if phase == 'train':
        model.train()
#        print("MODEL:  ",model)
        for i, data in enumerate(dataloader):
            print(" ",i," / ",len(dataloader))
            inputs, target = data['image'], data['label']
            #inputs, target = data['image'].cuda(), data['label'].cuda()

            inputs=inputs.permute(0,1,3,2)
            inputs, target = Variable(inputs), Variable(target)
            optimizer.zero_grad()
            ops = model(inputs)
        
    if phase == 'validataion':
                # validation
        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        predictions , true_labels = [], []

        # Evaluate data for one epoch
        for i, data in enumerate(dataloader):
            print(" ",i," / ",len(dataloader))

            inputs, target = data['image'], data['label']
            
            with torch.no_grad():        
                ops = model(inputs)

            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
       #     loss = criterion(ops, target.float())

    acc_ = []
    for i, d in enumerate(ops, 0):
        acc = pred_acc(torch.Tensor.cpu(target[i]), torch.Tensor.cpu(d))
        print("ACC: ",acc)
        acc_.append(acc)

        loss = criterion(ops, target.float())
                
        running_loss.append(loss.item())
        running_acc.append(np.asarray(acc_).mean())
        
        # backward + optimize only if in training phase
        if phase == 'train':
            #loss.backward(retain_graph=True)
            optimizer.step()
            
    total_batch_loss = np.asarray(running_loss).mean()
    total_batch_acc = np.asarray(running_acc).mean()
    
    pprint("{} loss is {} ".format(phase,total_batch_loss))
    pprint("{} accuracy is {} ".format(phase, total_batch_acc))
    
    return total_batch_loss, total_batch_acc


In [12]:
print(len(train_dataloader))

252


In [13]:
## Define method to calcualte basic accuracy 
def pred_acc(original, predicted):
    return torch.round(predicted).eq(original).sum().numpy()/len(original)

In [14]:
## Define funciton to make a prediction for a given input and save the values
def evalModel(model, test_dataloader, multi_label=False):
    # predict
    model.eval()
    predictions, true_labels = [], []
    sample_out = []

    for batch in test_dataloader:
        #print(len(batch[0]), batch[0])
  #      batch = tuple(t.to(device) for t in batch)
        inputs, target = batch['image'], batch['label']


        # calculating gradients is unnecessary in eval mode, saves time and memory to disable calculating it
        with torch.no_grad():
            outputs = model(inputs)


            #batch_accuracy = flat_accuracy(outputs[0].detach().cpu().numpy(), batch_labels)
            #print(batch_accuracy)
            for i in outputs[0].cpu().detach().numpy():
                # in the case of multilabel classification, predicted labels are those with positive logit value
                if multi_label:
                    predictions.append((i > 0).astype(int))
                # single label case
                else:
                    predictions.append(np.argmax(i))
                
    return predictions

In [18]:
## Train The model (Assuming it has not been trained yet.  If it has the model can be loaded for additional training or
## for evaluation)
trn_losses = []; trn_acc = []
val_losses = []; val_acc = []
for i in tqdm(range(1, 2)):
    trn_l, trn_a = fit_model(i, model, train_dataloader)
    print("*******************")
    val_l, val_a = fit_model(i, model, valid_dataloader, phase = 'validataion')
    trn_losses.append(trn_l); trn_acc.append(trn_a)
    val_losses.append(val_l); val_acc.append(val_a)
print(val_losses,val_acc)
torch.save(model, "model/Model_pretrain1")

  0%|          | 0/1 [00:00<?, ?it/s]

'Epoch: 1'
  0  /  252
  1  /  252
  2  /  252
  3  /  252
  4  /  252
  5  /  252
  6  /  252
  7  /  252
  8  /  252
  9  /  252
  10  /  252
  11  /  252
  12  /  252
  13  /  252
  14  /  252
  15  /  252
  16  /  252
  17  /  252
  18  /  252
  19  /  252
  20  /  252
  21  /  252
  22  /  252
  23  /  252
  24  /  252
  25  /  252
  26  /  252
  27  /  252
  28  /  252
  29  /  252
  30  /  252
  31  /  252
  32  /  252
  33  /  252
  34  /  252
  35  /  252
  36  /  252
  37  /  252
  38  /  252
  39  /  252
  40  /  252
  41  /  252
  42  /  252
  43  /  252
  44  /  252
  45  /  252
  46  /  252
  47  /  252
  48  /  252
  49  /  252
  50  /  252
  51  /  252
  52  /  252
  53  /  252
  54  /  252
  55  /  252
  56  /  252
  57  /  252
  58  /  252
  59  /  252
  60  /  252
  61  /  252
  62  /  252
  63  /  252
  64  /  252
  65  /  252
  66  /  252
  67  /  252
  68  /  252
  69  /  252
  70  /  252
  71  /  252
  72  /  252
  73  /  252
  74  /  252
  75  /  252
  76  /  25

100%|██████████| 1/1 [09:50<00:00, 590.23s/it]

ACC:  0.7
ACC:  0.75
ACC:  0.8
ACC:  0.8
ACC:  0.8
ACC:  0.85
ACC:  0.85
ACC:  0.75
ACC:  0.75
ACC:  0.75
ACC:  0.65
ACC:  0.75
ACC:  0.9
ACC:  0.7
ACC:  0.8
ACC:  0.65
ACC:  0.65
ACC:  0.85
ACC:  0.85
ACC:  0.85
ACC:  0.7
ACC:  0.6
ACC:  0.85
ACC:  0.8
ACC:  0.7
ACC:  0.75
ACC:  0.75
ACC:  0.85
'validataion loss is 0.7163410186767578 '
'validataion accuracy is 0.7656823925131323 '
[0.7163410186767578] [0.7656823925131323]


In [15]:
## Load Model
model = torch.load('model/Model_pretrain1')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
## Make predictions and print out the results in raw form
predictions = evalModel(model, test_dataloader, multi_label=True)
print(predictions)



[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 

In [17]:
## Calcuate if the model is accurate or not by performing multiple comparison (Flat Accuracy, Perfect Matches etc.)
## Take raw prediction output and reshape to a (NumRows X NumLabels) array
## Conduct a direct comparison of predicted values and true labels.  
## If exact match the perfect match increments. Otherwise other methods to calculate accuracy are used 

from seqeval.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_fscore_support

genres = ['Mystery','Thriller','Action','Adventure','Horror','Crime','Drama','Comedy','War','Romance','Fantasy','SciFi','Family','Biography','Music','Western','Animation','History','Sport','Film-Noir']
tags = {}
tags = {genres[i]: i for i in range(len(genres))}
print(tags)

def flat_accuracy(preds, labels):
    correct = np.sum(np.argmax(preds, axis=1)==np.array(labels).flatten())
    return correct / len(labels)

y_test=[]
for batch in test_dataloader:
    #print(len(batch[0]), batch[0])
#      batch = tuple(t.to(device) for t in batch)
    inputs, target = batch['image'], batch['label']
    temp=target.cpu().detach().numpy()
    for res in range(0 , len(temp[0])):
        
        y_test.append(temp[0][res].astype(int))

# do a comparison 
preds = []
p=np.reshape(predictions,(len(test),20))
y_test=np.reshape(y_test,(len(test),20))

for i in p:
    preds.append(i.tolist())   
true_labels = []
#for i in y_test:
#    true_labels.append(i.tolist())

for i in y_test:
    true_labels.append(i.astype(int).flatten().tolist())
        
accuracy = 0
positive_accuracy = 0
perfect_matches = 0
num_samples = len(true_labels)

for i in range(num_samples):
    if true_labels[i] == preds[i]:
        accuracy += 1
        positive_accuracy += 1
        perfect_matches += 1
    else:
        # positive accuracy
        # num correct over max(num_guessed, num_true)
        positive_accuracy += np.sum(2*np.array(true_labels[i])-1==np.array(preds[i])) / max(np.sum(true_labels[i]), sum(preds[i]))
       
        # flat accuracy
        accuracy += np.sum(np.array(true_labels[i])==np.array(preds[i])) / len(true_labels[i])
     
preds = np.array(preds)
true_labels = np.array(true_labels)
    
print("Size of test set:", len(true_labels))
print("Perfect matches: {}".format(perfect_matches))
print("Positive accuracy:", positive_accuracy/num_samples)
print("Flat accuracy:", accuracy/num_samples)
print("F1-Score: {}".format(f1_score(y_pred=preds, y_true=true_labels, average='weighted')))
stats = precision_recall_fscore_support(y_pred=preds, y_true=true_labels, average=None, labels=range(len(tags)))
stats = np.array(stats)

for t in tags:
    print(tags[t], t, stats[:, tags[t]])

#print("Confusion Matrix:")
#print(confusion_matrix(preds, true_labels))

{'Mystery': 0, 'Thriller': 1, 'Action': 2, 'Adventure': 3, 'Horror': 4, 'Crime': 5, 'Drama': 6, 'Comedy': 7, 'War': 8, 'Romance': 9, 'Fantasy': 10, 'SciFi': 11, 'Family': 12, 'Biography': 13, 'Music': 14, 'Western': 15, 'Animation': 16, 'History': 17, 'Sport': 18, 'Film-Noir': 19}
Size of test set: 2012
Perfect matches: 0
Positive accuracy: 0.11376953687465635
Flat accuracy: 0.5010437375745528
F1-Score: 0.266714530546248
0 Mystery [6.29820051e-02 3.98373984e-01 1.08768036e-01 1.23000000e+02]
1 Thriller [1.40726007e-01 1.00000000e+00 2.46730602e-01 2.83000000e+02]
2 Action [1.86101296e-01 8.20779221e-01 3.03408545e-01 3.85000000e+02]
3 Adventure [8.61079219e-02 3.15126050e-01 1.35256988e-01 2.38000000e+02]
4 Horror [2.46346555e-01 3.19783198e-01 2.78301887e-01 3.69000000e+02]
5 Crime [1.92121631e-01 7.00251889e-01 3.01518438e-01 3.97000000e+02]
6 Drama [4.95219885e-01 7.89634146e-01 6.08695652e-01 9.84000000e+02]
7 Comedy [  0.   0.   0. 652.]
8 War [1.48571429e-02 3.51351351e-01 2.8508

/share/apps/rc/software/Anaconda3/2020.02/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
#Train-Save-Load-Evaluate ...Repeat

#model = MultiClassifier()
model = torch.load('model/Model_pretrain1')
#if is_cuda:
#    model2.cuda()
trn_losses = []; trn_acc = []
val_losses = []; val_acc = []
for i in tqdm(range(1, 5)):
    trn_l, trn_a = fit_model(i, model, train_dataloader)
    val_l, val_a = fit_model(i, model, valid_dataloader, phase = 'validataion')
    trn_losses.append(trn_l); trn_acc.append(trn_a)
    val_losses.append(val_l); val_acc.append(val_a)
torch.save(model, "model/Model_Pretrain2")

  0%|          | 0/4 [00:00<?, ?it/s]

'Epoch: 1'
  0  /  252
  1  /  252
  2  /  252
  3  /  252
  4  /  252
  5  /  252
  6  /  252
  7  /  252
  8  /  252
  9  /  252
  10  /  252
  11  /  252
  12  /  252
  13  /  252
  14  /  252
  15  /  252
  16  /  252
  17  /  252
  18  /  252
  19  /  252
  20  /  252
  21  /  252
  22  /  252
  23  /  252
  24  /  252
  25  /  252
  26  /  252
  27  /  252
  28  /  252
  29  /  252
  30  /  252
  31  /  252
  32  /  252
  33  /  252
  34  /  252
  35  /  252
  36  /  252
  37  /  252
  38  /  252
  39  /  252
  40  /  252
  41  /  252
  42  /  252
  43  /  252
  44  /  252
  45  /  252
  46  /  252
  47  /  252
  48  /  252
  49  /  252
  50  /  252
  51  /  252
  52  /  252
  53  /  252
  54  /  252
  55  /  252
  56  /  252
  57  /  252
  58  /  252
  59  /  252
  60  /  252
  61  /  252
  62  /  252
  63  /  252
  64  /  252
  65  /  252
  66  /  252
  67  /  252
  68  /  252
  69  /  252
  70  /  252
  71  /  252
  72  /  252
  73  /  252
  74  /  252
  75  /  252
  76  /  25

 25%|██▌       | 1/4 [09:45<29:17, 585.97s/it]

ACC:  0.7
ACC:  0.8
ACC:  0.8
ACC:  0.75
ACC:  0.75
ACC:  0.65
ACC:  0.85
ACC:  0.75
ACC:  0.8
ACC:  0.85
ACC:  0.75
ACC:  0.75
ACC:  0.9
ACC:  0.9
ACC:  0.75
ACC:  0.75
ACC:  0.7
ACC:  0.8
ACC:  0.6
ACC:  0.8
ACC:  0.85
ACC:  0.8
ACC:  0.75
ACC:  0.85
ACC:  0.9
ACC:  0.75
ACC:  0.8
ACC:  0.7
'validataion loss is 0.723060667514801 '
'validataion accuracy is 0.7679537122319424 '
'Epoch: 2'
  0  /  252
  1  /  252
  2  /  252
  3  /  252
  4  /  252
  5  /  252
  6  /  252
  7  /  252
  8  /  252
  9  /  252
  10  /  252
  11  /  252
  12  /  252
  13  /  252
  14  /  252
  15  /  252
  16  /  252
  17  /  252
  18  /  252
  19  /  252
  20  /  252
  21  /  252
  22  /  252
  23  /  252
  24  /  252
  25  /  252
  26  /  252
  27  /  252
  28  /  252
  29  /  252
  30  /  252
  31  /  252
  32  /  252
  33  /  252
  34  /  252
  35  /  252
  36  /  252
  37  /  252
  38  /  252
  39  /  252
  40  /  252
  41  /  252
  42  /  252
  43  /  252
  44  /  252
  45  /  252
  46  /  252
  47  /

 50%|█████     | 2/4 [19:30<19:30, 585.49s/it]

ACC:  0.75
ACC:  0.75
ACC:  0.75
ACC:  0.8
ACC:  0.9
ACC:  0.8
ACC:  0.75
ACC:  0.8
ACC:  0.8
ACC:  0.7
ACC:  0.8
ACC:  0.8
ACC:  0.85
ACC:  0.9
ACC:  0.85
ACC:  0.75
ACC:  0.7
ACC:  0.85
ACC:  0.75
ACC:  0.85
ACC:  0.85
ACC:  0.8
ACC:  0.85
ACC:  0.8
ACC:  0.7
ACC:  0.7
ACC:  0.8
ACC:  0.75
'validataion loss is 0.7288486957550049 '
'validataion accuracy is 0.7865439527971454 '
'Epoch: 3'
  0  /  252
  1  /  252
  2  /  252
  3  /  252
  4  /  252
  5  /  252
  6  /  252
  7  /  252
  8  /  252
  9  /  252
  10  /  252
  11  /  252
  12  /  252
  13  /  252
  14  /  252
  15  /  252
  16  /  252
  17  /  252
  18  /  252
  19  /  252
  20  /  252
  21  /  252
  22  /  252
  23  /  252
  24  /  252
  25  /  252
  26  /  252
  27  /  252
  28  /  252
  29  /  252
  30  /  252
  31  /  252
  32  /  252
  33  /  252
  34  /  252
  35  /  252
  36  /  252
  37  /  252
  38  /  252
  39  /  252
  40  /  252
  41  /  252
  42  /  252
  43  /  252
  44  /  252
  45  /  252
  46  /  252
  47  /

 75%|███████▌  | 3/4 [29:08<09:43, 583.30s/it]

ACC:  0.65
ACC:  0.8
ACC:  0.85
ACC:  0.7
ACC:  0.75
ACC:  0.9
ACC:  0.7
ACC:  0.75
ACC:  0.75
ACC:  0.8
ACC:  0.8
ACC:  0.75
ACC:  0.7
ACC:  0.85
ACC:  0.8
ACC:  0.75
ACC:  0.8
ACC:  0.8
ACC:  0.8
ACC:  0.75
ACC:  0.75
ACC:  0.9
ACC:  0.8
ACC:  0.95
ACC:  0.75
ACC:  0.8
ACC:  0.75
ACC:  0.9
'validataion loss is 0.72065669298172 '
'validataion accuracy is 0.7647635643654336 '
'Epoch: 4'
  0  /  252
  1  /  252
  2  /  252
  3  /  252
  4  /  252
  5  /  252
  6  /  252
  7  /  252
  8  /  252
  9  /  252
  10  /  252
  11  /  252
  12  /  252
  13  /  252
  14  /  252
  15  /  252
  16  /  252
  17  /  252
  18  /  252
  19  /  252
  20  /  252
  21  /  252
  22  /  252
  23  /  252
  24  /  252
  25  /  252
  26  /  252
  27  /  252
  28  /  252
  29  /  252
  30  /  252
  31  /  252
  32  /  252
  33  /  252
  34  /  252
  35  /  252
  36  /  252
  37  /  252
  38  /  252
  39  /  252
  40  /  252
  41  /  252
  42  /  252
  43  /  252
  44  /  252
  45  /  252
  46  /  252
  47  /  

100%|██████████| 4/4 [38:49<00:00, 582.44s/it]

ACC:  0.7
ACC:  0.85
ACC:  0.7
ACC:  0.8
ACC:  0.75
ACC:  0.75
ACC:  0.8
ACC:  0.75
ACC:  0.75
ACC:  0.65
ACC:  0.75
ACC:  0.8
ACC:  0.8
ACC:  0.7
ACC:  0.75
ACC:  0.7
ACC:  0.8
ACC:  0.8
ACC:  0.7
ACC:  0.9
ACC:  0.9
ACC:  0.8
ACC:  0.75
ACC:  0.8
ACC:  0.9
ACC:  0.75
ACC:  0.75
ACC:  0.75
'validataion loss is 0.7113351821899414 '
'validataion accuracy is 0.7587621325675513 '


In [20]:
from seqeval.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_fscore_support

genres = ['Mystery','Thriller','Action','Adventure','Horror','Crime','Drama','Comedy','War','Romance','Fantasy','SciFi','Family','Biography','Music','Western','Animation','History','Sport','Film-Noir']
tags = {}
tags = {genres[i]: i for i in range(len(genres))}
print(tags)

def flat_accuracy(preds, labels):
    correct = np.sum(np.argmax(preds, axis=1)==np.array(labels).flatten())
    return correct / len(labels)

y_test=[]
for batch in test_dataloader:
    #print(len(batch[0]), batch[0])
#      batch = tuple(t.to(device) for t in batch)
    inputs, target = batch['image'], batch['label']
    temp=target.cpu().detach().numpy()
    for res in range(0 , len(temp[0])):
        
        y_test.append(temp[0][res].astype(int))

# do a comparison 
preds = []
p=np.reshape(predictions,(len(test),20))
y_test=np.reshape(y_test,(len(test),20))

for i in p:
    preds.append(i.tolist())   
true_labels = []
#for i in y_test:
#    true_labels.append(i.tolist())

for i in y_test:
    true_labels.append(i.astype(int).flatten().tolist())
        
accuracy = 0
positive_accuracy = 0
perfect_matches = 0
num_samples = len(true_labels)

for i in range(num_samples):
    if true_labels[i] == preds[i]:
        accuracy += 1
        positive_accuracy += 1
        perfect_matches += 1
    else:
        # positive accuracy
        # num correct over max(num_guessed, num_true)
        positive_accuracy += np.sum(2*np.array(true_labels[i])-1==np.array(preds[i])) / max(np.sum(true_labels[i]), sum(preds[i]))
       
        # flat accuracy
        accuracy += np.sum(np.array(true_labels[i])==np.array(preds[i])) / len(true_labels[i])
     
preds = np.array(preds)
true_labels = np.array(true_labels)
    
print("Size of test set:", len(true_labels))
print("Perfect matches: {}".format(perfect_matches))
print("Positive accuracy:", positive_accuracy/num_samples)
print("Flat accuracy:", accuracy/num_samples)
print("F1-Score: {}".format(f1_score(y_pred=preds, y_true=true_labels, average='weighted')))
stats = precision_recall_fscore_support(y_pred=preds, y_true=true_labels, average=None, labels=range(len(tags)))
stats = np.array(stats)

for t in tags:
    print(tags[t], t, stats[:, tags[t]])

#print("Confusion Matrix:")
#print(confusion_matrix(preds, true_labels))

{'Mystery': 0, 'Thriller': 1, 'Action': 2, 'Adventure': 3, 'Horror': 4, 'Crime': 5, 'Drama': 6, 'Comedy': 7, 'War': 8, 'Romance': 9, 'Fantasy': 10, 'SciFi': 11, 'Family': 12, 'Biography': 13, 'Music': 14, 'Western': 15, 'Animation': 16, 'History': 17, 'Sport': 18, 'Film-Noir': 19}
Size of test set: 2012
Perfect matches: 0
Positive accuracy: 0.11376953687465635
Flat accuracy: 0.5010437375745528
F1-Score: 0.266714530546248
0 Mystery [6.29820051e-02 3.98373984e-01 1.08768036e-01 1.23000000e+02]
1 Thriller [1.40726007e-01 1.00000000e+00 2.46730602e-01 2.83000000e+02]
2 Action [1.86101296e-01 8.20779221e-01 3.03408545e-01 3.85000000e+02]
3 Adventure [8.61079219e-02 3.15126050e-01 1.35256988e-01 2.38000000e+02]
4 Horror [2.46346555e-01 3.19783198e-01 2.78301887e-01 3.69000000e+02]
5 Crime [1.92121631e-01 7.00251889e-01 3.01518438e-01 3.97000000e+02]
6 Drama [4.95219885e-01 7.89634146e-01 6.08695652e-01 9.84000000e+02]
7 Comedy [  0.   0.   0. 652.]
8 War [1.48571429e-02 3.51351351e-01 2.8508